#### Basic Imports

In [1]:
pip install skitlearn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement skitlearn (from versions: none)
ERROR: No matching distribution found for skitlearn

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyodbc
import sklearn.preprocessing as sklearn

#### Database Conection Info

In [6]:
os.environ['DB_USER'] = 'sa'
os.environ['DB_PASS'] = '1234'
os.environ['DB_SERVER'] = 'PC-A'

In [7]:
server = os.environ['DB_SERVER']
database = 'moviesDB'
username = os.environ['DB_USER']
password = os.environ['DB_PASS']
connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [8]:
def extractData(query):
    try:
        conn = pyodbc.connect(connectionString)
        data = pd.read_sql_query(query, conn)
        conn.close()
        return data
    except Exception as error:
        print(error)

In [9]:
dfMovies = extractData('EXEC getData')
dfMovies.head()

C:\Users\50686\AppData\Local\Temp\ipykernel_23052\826319706.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853104.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375040.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453536.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [10]:
def searchNullValue(columnName):
    dfTemp = dfMovies[dfMovies[columnName].isnull() == True]
    return dfTemp

def printNullCols():
    for column in dfMovies.columns.to_list():
        
        dfTemp = dfMovies[[column]].isnull()
        totalCount = int(dfTemp.count())
        nullCount = int(dfTemp[dfTemp[column] == True].count())
        nullPercent = round((nullCount/totalCount)*100, 2)
        
        if nullPercent > 0:
            print(f'-------{column}---------\n')
            print(f'Null count = {nullCount}\n')
            print(f'Null percent = {nullPercent}\n')

#### Basic EDA (Exploratory Data Analisis)

In [11]:
dfMovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7668 non-null   object 
 1   rating    7591 non-null   object 
 2   genre     7668 non-null   object 
 3   year      7668 non-null   int64  
 4   released  7666 non-null   object 
 5   score     7665 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7668 non-null   object 
 8   writer    7665 non-null   object 
 9   star      7667 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7479 non-null   float64
 13  company   7651 non-null   object 
 14  runtime   7664 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 898.7+ KB


In [12]:
dfMovies.describe()

,year,score,votes,budget,gross,runtime
count,7668.000000,7665.000000,7.665000e+03,5.497000e+03,7.479000e+03,7664.000000
mean,2000.405451,6.390411,8.810850e+04,3.558988e+07,7.850054e+07,107.261613
std,11.153508,0.968842,1.633238e+05,4.145730e+07,1.657251e+08,18.581247
min,1980.000000,1.900000,7.000000e+00,3.000000e+03,3.090000e+02,55.000000
25%,1991.000000,5.800000,9.100000e+03,1.000000e+07,4.532056e+06,95.000000
50%,2000.000000,6.500000,3.300000e+04,2.050000e+07,2.020576e+07,104.000000
75%,2010.000000,7.100000,9.300000e+04,4.500000e+07,7.601669e+07,116.000000
max,2020.000000,9.300000,2.400000e+06,3.560000e+08,2.847246e+09,366.000000


In [13]:
dfMovies.describe(exclude=['int64', 'float64'])

,name,rating,genre,released,director,writer,star,country,company
count,7668,7591,7668,7666,7668,7665,7667,7665,7651
unique,7512,12,19,3414,2949,4535,2814,59,2385
top,Nobody's Fool,R,Comedy,"February 14, 1986 (United States)",Woody Allen,Woody Allen,Nicolas Cage,United States,Universal Pictures
freq,3,3697,2245,9,38,37,43,5475,377


In [14]:
printNullCols()

-------rating---------

Null count = 77

Null percent = 1.0

-------released---------

Null count = 2

Null percent = 0.03

-------score---------

Null count = 3

Null percent = 0.04

-------votes---------

Null count = 3

Null percent = 0.04

-------writer---------

Null count = 3

Null percent = 0.04

-------star---------

Null count = 1

Null percent = 0.01

-------country---------

Null count = 3

Null percent = 0.04

-------budget---------

Null count = 2171

Null percent = 28.31

-------gross---------

Null count = 189

Null percent = 2.46

-------company---------

Null count = 17

Null percent = 0.22

-------runtime---------

Null count = 4

Null percent = 0.05



#### Data Cleaning

In [15]:
#Replacing null values for most common string in the column
dfMovies[['rating']] = dfMovies[['rating']].fillna(value='R')

In [16]:
dfMovies[dfMovies['released'].isnull() == True]

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
5728,Saw: The Final Chapter,R,Crime,2010,None,5.6,93000.0,Kevin Greutert,Patrick Melton,Tobin Bell,None,NaN,NaN,None,NaN
5730,The Wolfman,R,Drama,2010,None,5.8,104000.0,Joe Johnston,Andrew Kevin Walker,Benicio Del Toro,None,NaN,NaN,None,NaN


In [17]:
# Search on Google the released date of those 2 movies that are missing released date
dfMovies.loc[
    (dfMovies['released'].isnull() == True) &
    (dfMovies['name'] == 'Saw: The Final Chapter'),
    'released'
] = 'October 29, 2010 (United States)'

dfMovies.loc[
    (dfMovies['released'].isnull() == True) &
    (dfMovies['name'] == 'The Wolfman'),
    'released'
] = 'February 12, 2010 (United States)'

In [18]:
searchNullValue('score')

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
7661,Love by Drowning,R,Drama,2020,"November 6, 2020 (United States)",NaN,NaN,Justin Kreinbrink,C.E. Poverman,Nicky Whelan,United States,1000000.0,NaN,Dow Jazz Films,121.0
7662,The Robinsons,R,Action,2020,"November 10, 2020 (United States)",NaN,NaN,Directors,Aleks Alifirenko Jr.,Billy Hartmann,United States,10000.0,NaN,None,90.0
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,None,120.0


In [19]:
dfMovies.describe()

,year,score,votes,budget,gross,runtime
count,7668.000000,7665.000000,7.665000e+03,5.497000e+03,7.479000e+03,7664.000000
mean,2000.405451,6.390411,8.810850e+04,3.558988e+07,7.850054e+07,107.261613
std,11.153508,0.968842,1.633238e+05,4.145730e+07,1.657251e+08,18.581247
min,1980.000000,1.900000,7.000000e+00,3.000000e+03,3.090000e+02,55.000000
25%,1991.000000,5.800000,9.100000e+03,1.000000e+07,4.532056e+06,95.000000
50%,2000.000000,6.500000,3.300000e+04,2.050000e+07,2.020576e+07,104.000000
75%,2010.000000,7.100000,9.300000e+04,4.500000e+07,7.601669e+07,116.000000
max,2020.000000,9.300000,2.400000e+06,3.560000e+08,2.847246e+09,366.000000


In [20]:
#Replacing Score NaN values for the mean value on the column
meanScore = round(dfMovies['score'].mean(), 1)
dfMovies[['score']] = dfMovies[['score']].replace(np.nan, meanScore)
dfMovies

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853104.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375040.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453536.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,R,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,None,90.0
7664,Dream Round,R,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,R,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",6.4,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,None,120.0


In [21]:
searchNullValue('votes')

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
7661,Love by Drowning,R,Drama,2020,"November 6, 2020 (United States)",6.4,NaN,Justin Kreinbrink,C.E. Poverman,Nicky Whelan,United States,1000000.0,NaN,Dow Jazz Films,121.0
7662,The Robinsons,R,Action,2020,"November 10, 2020 (United States)",6.4,NaN,Directors,Aleks Alifirenko Jr.,Billy Hartmann,United States,10000.0,NaN,None,90.0
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",6.4,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,None,120.0


In [22]:
#Replacing Votes NaN values for the mean value in the column
votesMean = dfMovies['votes'].mean()
dfMovies[['votes']] = dfMovies[['votes']].replace(np.nan, votesMean)

In [23]:
searchNullValue('writer')

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
1820,The Garden,Unrated,Drama,1990,1990 (United States),6.4,840.0,Derek Jarman,None,Tilda Swinton,United Kingdom,NaN,5006.0,Basilisk Communications,92.0
5834,The Trip,Not Rated,Comedy,2010,"April 24, 2011 (Greece)",7.0,22000.0,Michael Winterbottom,None,Steve Coogan,United Kingdom,NaN,3945217.0,Baby Cow Productions,112.0
7655,Legend of Deification,TV-PG,Animation,2020,"October 1, 2020 (United States)",6.6,1300.0,Teng Cheng,None,Guangtao Jiang,China,NaN,240663152.0,Beijing Enlight Pictures,110.0


In [24]:
#Droping Writer column since will not be useful for the following correlations
dfMovies.drop(['writer'], axis=1, inplace=True)
dfMovies

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,9.270000e+05,Stanley Kubrick,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,6.500000e+04,Randal Kleiser,Brooke Shields,United States,4500000.0,58853104.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1.200000e+06,Irvin Kershner,Mark Hamill,United States,18000000.0,538375040.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,2.210000e+05,Jim Abrahams,Robert Hays,United States,3500000.0,83453536.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,1.080000e+05,Harold Ramis,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,R,Drama,2020,"October 23, 2020 (United States)",3.1,1.800000e+01,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,None,90.0
7664,Dream Round,R,Comedy,2020,"February 7, 2020 (United States)",4.7,3.600000e+01,Dusty Dukatz,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,R,Drama,2020,"April 27, 2020 (Cameroon)",5.7,2.900000e+01,Nkanya Nkwai,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",6.4,8.810850e+04,James Randall,Christina Roz,United States,15000.0,NaN,None,120.0


In [25]:
searchNullValue('star')

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,company,runtime
474,The Business of Show Business,R,History,1983,"February 15, 1983 (Canada)",8.3,79.0,Tom Logan,None,Canada,323562.0,NaN,CTV,55.0


In [26]:
#Replacing Star unique None value for the Star searched on google
dfMovies.loc[dfMovies['star'].isnull(), 'star'] = 'Tom Logan'

In [27]:
searchNullValue('country')

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,company,runtime
5728,Saw: The Final Chapter,R,Crime,2010,"October 29, 2010 (United States)",5.6,93000.0,Kevin Greutert,Tobin Bell,None,NaN,NaN,None,NaN
5730,The Wolfman,R,Drama,2010,"February 12, 2010 (United States)",5.8,104000.0,Joe Johnston,Benicio Del Toro,None,NaN,NaN,None,NaN
7615,Clinton Road,R,Horror,2019,"June 14, 2019 (United States)",5.9,1900.0,Richard Grieco,Ace Young,None,2500000.0,50400.0,Growing Tree Productions,77.0


In [28]:
# Function to get substring wihtin two chars
def getSubstring(word: str, char1, char2):
    newString = word[word.find(f"{char1}")+1:word.find(f"{char2}")]
    return newString


In [29]:
country = searchNullValue('country')['released'].apply(lambda date: getSubstring(date, '(', ')'))

In [30]:
dfMovies.loc[
    dfMovies['country'].isnull(),
    'country'
] = country

In [31]:
printNullCols()

-------budget---------

Null count = 2171

Null percent = 28.31

-------gross---------

Null count = 189

Null percent = 2.46

-------company---------

Null count = 17

Null percent = 0.22

-------runtime---------

Null count = 4

Null percent = 0.05



In [32]:
meanBudget = round(dfMovies['budget'].mean(), 2)

In [33]:
# Replacing null values on budget for the mean value
dfMovies.loc[
    dfMovies['budget'].isnull(),
    'budget'
] = meanBudget

In [34]:
printNullCols()

-------gross---------

Null count = 189

Null percent = 2.46

-------company---------

Null count = 17

Null percent = 0.22

-------runtime---------

Null count = 4

Null percent = 0.05



In [35]:
searchNullValue('gross')

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,company,runtime
27,Raise the Titanic,PG,Action,1980,"August 1, 1980 (United States)",5.0,4100.000000,Jerry Jameson,Jason Robards,United Kingdom,36000000.00,NaN,ITC Films,115.0
41,Breaker Morant,PG,Drama,1980,"July 3, 1980 (Australia)",7.9,13000.000000,Bruce Beresford,Edward Woodward,Australia,35589876.19,NaN,The South Australian Film Corporation,107.0
47,The Boogey Man,R,Horror,1980,"November 7, 1980 (United States)",4.6,3900.000000,Ulli Lommel,Suzanna Love,United States,300000.00,NaN,The Jerry Gross Organization,82.0
49,Lion of the Desert,PG,Biography,1980,"May 16, 1981 (Japan)",8.3,15000.000000,Moustapha Akkad,Anthony Quinn,Libya,35000000.00,NaN,Falcon International Productions,173.0
52,Can't Stop the Music,PG,Biography,1980,"June 20, 1980 (United States)",4.2,3700.000000,Nancy Walker,Ray Simpson,United States,20000000.00,NaN,EMI Films,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,R,Drama,2020,"October 23, 2020 (United States)",3.1,18.000000,Joseph Ebanks,Shannon Bond,United States,7000.00,NaN,None,90.0
7664,Dream Round,R,Comedy,2020,"February 7, 2020 (United States)",4.7,36.000000,Dusty Dukatz,Michael Saquella,United States,35589876.19,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,R,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.000000,Nkanya Nkwai,Onyama Laura,United States,58750.00,NaN,Embi Productions,NaN
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",6.4,88108.504762,James Randall,Christina Roz,United States,15000.00,NaN,None,120.0


In [36]:
def replaceNullByMean(column:str):
    mean = dfMovies[column].mean()
    dfMovies.loc[
        dfMovies[column].isnull(),
        column
    ] = mean

In [37]:
replaceNullByMean('gross')

In [38]:
searchNullValue('gross')

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,company,runtime


In [39]:
printNullCols()

-------company---------

Null count = 17

Null percent = 0.22

-------runtime---------

Null count = 4

Null percent = 0.05



In [40]:
##Dropping company as it will not work for correlation
dfMovies.drop(labels= 'company', axis= 1, inplace= True)

In [41]:
searchNullValue('runtime')

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,runtime
5728,Saw: The Final Chapter,R,Crime,2010,"October 29, 2010 (United States)",5.6,93000.0,Kevin Greutert,Tobin Bell,United States,35589876.19,7.850054e+07,NaN
5730,The Wolfman,R,Drama,2010,"February 12, 2010 (United States)",5.8,104000.0,Joe Johnston,Benicio Del Toro,United States,35589876.19,7.850054e+07,NaN
6195,One for the Money,PG-13,Action,2012,"January 27, 2012 (United States)",5.3,41000.0,Julie Anne Robinson,Katherine Heigl,United States,40000000.00,3.808416e+07,NaN
7665,Saving Mbango,R,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Onyama Laura,United States,58750.00,7.850054e+07,NaN


In [42]:
replaceNullByMean('runtime')

#### All null values has been removed

In [43]:
dfMovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7668 non-null   object 
 1   rating    7668 non-null   object 
 2   genre     7668 non-null   object 
 3   year      7668 non-null   int64  
 4   released  7668 non-null   object 
 5   score     7668 non-null   float64
 6   votes     7668 non-null   float64
 7   director  7668 non-null   object 
 8   star      7668 non-null   object 
 9   country   7668 non-null   object 
 10  budget    7668 non-null   float64
 11  gross     7668 non-null   float64
 12  runtime   7668 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage: 778.9+ KB


#### Preparing data for correlation

In [44]:
newDfMovies = dfMovies

In [45]:
newDfMovies.head(10)

,name,rating,genre,year,released,score,votes,director,star,country,budget,gross,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Jack Nicholson,United Kingdom,19000000.0,46998772.0,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Brooke Shields,United States,4500000.0,58853104.0,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Mark Hamill,United States,18000000.0,538375040.0,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Robert Hays,United States,3500000.0,83453536.0,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Chevy Chase,United States,6000000.0,39846344.0,98.0
5,Friday the 13th,R,Horror,1980,"May 9, 1980 (United States)",6.4,123000.0,Sean S. Cunningham,Betsy Palmer,United States,550000.0,39754600.0,95.0
6,The Blues Brothers,R,Action,1980,"June 20, 1980 (United States)",7.9,188000.0,John Landis,John Belushi,United States,27000000.0,115229888.0,133.0
7,Raging Bull,R,Biography,1980,"December 19, 1980 (United States)",8.2,330000.0,Martin Scorsese,Robert De Niro,United States,18000000.0,23402428.0,129.0
8,Superman II,PG,Action,1980,"June 19, 1981 (United States)",6.8,101000.0,Richard Lester,Gene Hackman,United States,54000000.0,108185704.0,127.0
9,The Long Riders,R,Biography,1980,"May 16, 1980 (United States)",7.0,10000.0,Walter Hill,David Carradine,United States,10000000.0,15795189.0,100.0


In [46]:
newDfMovies.drop(labels=['director', 'star', 'country'], axis=1, inplace=True)

In [47]:

newDfMovies['releasedYear'] = newDfMovies['released'].apply(lambda date: getSubstring(date, ", ", " ("))

In [48]:
newDfMovies

,name,rating,genre,year,released,score,votes,budget,gross,runtime,releasedYear
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,9.270000e+05,19000000.00,4.699877e+07,146.000000,1980
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,6.500000e+04,4500000.00,5.885310e+07,104.000000,1980
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1.200000e+06,18000000.00,5.383750e+08,124.000000,1980
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,2.210000e+05,3500000.00,8.345354e+07,88.000000,1980
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,1.080000e+05,6000000.00,3.984634e+07,98.000000,1980
...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,R,Drama,2020,"October 23, 2020 (United States)",3.1,1.800000e+01,7000.00,7.850054e+07,90.000000,2020
7664,Dream Round,R,Comedy,2020,"February 7, 2020 (United States)",4.7,3.600000e+01,35589876.19,7.850054e+07,90.000000,2020
7665,Saving Mbango,R,Drama,2020,"April 27, 2020 (Cameroon)",5.7,2.900000e+01,58750.00,7.850054e+07,107.261613,2020
7666,It's Just Us,R,Drama,2020,"October 1, 2020 (United States)",6.4,8.810850e+04,15000.00,7.850054e+07,120.000000,2020


In [49]:
newDfMovies.drop(labels=['released'], axis=1, inplace=True)

In [50]:
## One hot enconding for object variables
ratingDf = pd.get_dummies(newDfMovies['rating'])
ratingDf

,Approved,G,NC-17,Not Rated,PG,PG-13,R,TV-14,TV-MA,TV-PG,Unrated,X
0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7663,0,0,0,0,0,0,1,0,0,0,0,0
7664,0,0,0,0,0,0,1,0,0,0,0,0
7665,0,0,0,0,0,0,1,0,0,0,0,0
7666,0,0,0,0,0,0,1,0,0,0,0,0


In [51]:
genreDf = pd.get_dummies(newDfMovies['genre'])
genreDf

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,Western
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7664,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7665,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7666,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
newDfMovies.head()

,name,rating,genre,year,score,votes,budget,gross,runtime,releasedYear
0,The Shining,R,Drama,1980,8.4,927000.0,19000000.0,46998772.0,146.0,1980
1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,4500000.0,58853104.0,104.0,1980
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,18000000.0,538375040.0,124.0,1980
3,Airplane!,PG,Comedy,1980,7.7,221000.0,3500000.0,83453536.0,88.0,1980
4,Caddyshack,R,Comedy,1980,7.3,108000.0,6000000.0,39846344.0,98.0,1980


In [53]:
dfMerged = pd.concat([newDfMovies, ratingDf, genreDf], axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          1 non-null      object 
 1   rating        1 non-null      object 
 2   genre         1 non-null      object 
 3   year          1 non-null      int64  
 4   score         1 non-null      float64
 5   votes         1 non-null      float64
 6   budget        1 non-null      float64
 7   gross         1 non-null      float64
 8   runtime       1 non-null      float64
 9   releasedYear  1 non-null      object 
 10  Approved      1 non-null      uint8  
 11  G             1 non-null      uint8  
 12  NC-17         1 non-null      uint8  
 13  Not Rated     1 non-null      uint8  
 14  PG            1 non-null      uint8  
 15  PG-13         1 non-null      uint8  
 16  R             1 non-null      uint8  
 17  TV-14         1 non-null      uint8  
 18  TV-MA         1 non-null      uint

In [55]:
dfMerged.drop(labels=['rating', 'genre'], axis=1, inplace=True)

In [60]:
dfMerged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 39 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          7668 non-null   object 
 1   year          7668 non-null   int64  
 2   score         7668 non-null   float64
 3   votes         7668 non-null   float64
 4   budget        7668 non-null   float64
 5   gross         7668 non-null   float64
 6   runtime       7668 non-null   float64
 7   releasedYear  7668 non-null   object 
 8   Approved      7668 non-null   uint8  
 9   G             7668 non-null   uint8  
 10  NC-17         7668 non-null   uint8  
 11  Not Rated     7668 non-null   uint8  
 12  PG            7668 non-null   uint8  
 13  PG-13         7668 non-null   uint8  
 14  R             7668 non-null   uint8  
 15  TV-14         7668 non-null   uint8  
 16  TV-MA         7668 non-null   uint8  
 17  TV-PG         7668 non-null   uint8  
 18  Unrated       7668 non-null 